# DuckDB
Tips and tricks for DuckDB


# Read on disk
Sometimes you may need to work with big csv files that don't fit in memory. With DuckDB you can read the file on disk and query it without loading it in memory. Below is an example of how to do it.




In [10]:
import duckdb
query = """
SELECT * FROM read_csv('https://raw.githubusercontent.com/TorOEkle/skeleton_project/refs/heads/main/duckdb/data/stocks.csv', delim = ',')
WHERE timestamp < '2023-03-01'
"""

with duckdb.connect() as con:
    df = con.execute(query).fetchdf()
con.close()
df

,timestamp,stock_ticket,stock_value,holding
0,2023-01-01,AAPL,150.25,10
1,2023-01-15,AAPL,152.30,10
2,2023-02-01,AAPL,148.90,12
3,2023-02-15,AAPL,151.00,12
4,2023-01-01,GOOGL,2800.50,5
5,2023-01-15,GOOGL,2825.75,5
6,2023-02-01,GOOGL,2790.30,6
7,2023-02-15,GOOGL,2810.50,6
8,2023-01-01,MSFT,300.25,20
9,2023-01-15,MSFT,305.30,20


In [11]:
query = """
SELECT dividends.*, 
    stocks.holding , 
    dividends.amount * stocks.holding as dividend_amount
FROM read_csv('https://raw.githubusercontent.com/TorOEkle/skeleton_project/refs/heads/main/duckdb/data/dividends.csv', delim = ',') as dividends
ASOF LEFT JOIN read_csv('https://raw.githubusercontent.com/TorOEkle/skeleton_project/refs/heads/main/duckdb/data/stocks.csv', delim = ',') as stocks 
    ON stocks.stock_ticket = dividends.stock_ticket
    AND stocks.timestamp <= dividends.timestamp

"""

with duckdb.connect() as con:
    df = con.execute(query).fetchdf()
con.close()
df

,timestamp,stock_ticket,amount,holding,dividend_amount
0,2023-03-20,AAPL,12.80,20,256.0
1,2023-09-20,AAPL,20.85,30,625.5
2,2023-03-20,MSFT,2.20,25,55.0
3,2023-09-20,MSFT,6.25,40,250.0
4,2023-03-20,GOOGL,5.50,8,44.0
5,2023-09-20,GOOGL,4.60,10,46.0
